In [1]:
import os
import tensorflow as tf


from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2021-03-17 06:11:48--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.11.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.11.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   161MB/s    in 0.5s    

2021-03-17 06:11:49 (161 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [2]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (8, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [3]:
!wget --no-check-certificate \
    https://datasets.simula.no/kvasir/data/kvasir-dataset-v2.zip \
    -O /tmp/kvasir-dataset-v2.zip

--2021-03-17 06:11:56--  https://datasets.simula.no/kvasir/data/kvasir-dataset-v2.zip
Resolving datasets.simula.no (datasets.simula.no)... 128.39.36.14
Connecting to datasets.simula.no (datasets.simula.no)|128.39.36.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2489312085 (2.3G) [application/zip]
Saving to: ‘/tmp/kvasir-dataset-v2.zip’

/tmp/kvasir-dataset 100%[===================>]   2.32G  23.1MB/s    in 1m 44s  

2021-03-17 06:13:41 (22.7 MB/s) - ‘/tmp/kvasir-dataset-v2.zip’ saved [2489312085/2489312085]



In [4]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

# file_path = '/tmp/kvasir-dataset-v2/'
# os.remove(file_path)

local_zip = '/tmp/kvasir-dataset-v2.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

from glob import glob
glob("/tmp/kvasir-dataset-v2/*/")

['/tmp/kvasir-dataset-v2/normal-z-line/',
 '/tmp/kvasir-dataset-v2/ulcerative-colitis/',
 '/tmp/kvasir-dataset-v2/esophagitis/',
 '/tmp/kvasir-dataset-v2/normal-cecum/',
 '/tmp/kvasir-dataset-v2/dyed-lifted-polyps/',
 '/tmp/kvasir-dataset-v2/dyed-resection-margins/',
 '/tmp/kvasir-dataset-v2/normal-pylorus/',
 '/tmp/kvasir-dataset-v2/polyps/']

In [5]:
def fast_scandir(dirname):
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

#os.mkdir('/tmp/kvasir-dataset-v2/data/training/dyed-lifted-polyps/')
def mkdirectory():
    try:
      os.mkdir('/tmp/kvasir-dataset-v2/data/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing')

      os.mkdir('/tmp/kvasir-dataset-v2/data/training/dyed-lifted-polyps/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/dyed-resection-margins/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/esophagitis/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/normal-cecum/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/normal-pylorus/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/normal-z-line/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/polyps/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/training/ulcerative-colitis/')
      
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/dyed-lifted-polyps/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/dyed-resection-margins/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/esophagitis/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/normal-cecum/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/normal-pylorus/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/normal-z-line/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/polyps/')
      os.mkdir('/tmp/kvasir-dataset-v2/data/testing/ulcerative-colitis/')
      
    except OSError:
      pass

mkdirectory()
fast_scandir('/tmp/kvasir-dataset-v2/')

# from glob import glob
# glob("/tmp/kvasir-dataset-v2/*/")

['/tmp/kvasir-dataset-v2/data',
 '/tmp/kvasir-dataset-v2/normal-z-line',
 '/tmp/kvasir-dataset-v2/ulcerative-colitis',
 '/tmp/kvasir-dataset-v2/esophagitis',
 '/tmp/kvasir-dataset-v2/normal-cecum',
 '/tmp/kvasir-dataset-v2/dyed-lifted-polyps',
 '/tmp/kvasir-dataset-v2/dyed-resection-margins',
 '/tmp/kvasir-dataset-v2/normal-pylorus',
 '/tmp/kvasir-dataset-v2/polyps',
 '/tmp/kvasir-dataset-v2/data/testing',
 '/tmp/kvasir-dataset-v2/data/training',
 '/tmp/kvasir-dataset-v2/data/testing/normal-z-line',
 '/tmp/kvasir-dataset-v2/data/testing/ulcerative-colitis',
 '/tmp/kvasir-dataset-v2/data/testing/esophagitis',
 '/tmp/kvasir-dataset-v2/data/testing/normal-cecum',
 '/tmp/kvasir-dataset-v2/data/testing/dyed-lifted-polyps',
 '/tmp/kvasir-dataset-v2/data/testing/dyed-resection-margins',
 '/tmp/kvasir-dataset-v2/data/testing/normal-pylorus',
 '/tmp/kvasir-dataset-v2/data/testing/polyps',
 '/tmp/kvasir-dataset-v2/data/training/normal-z-line',
 '/tmp/kvasir-dataset-v2/data/training/ulcerative-co

In [6]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


SOURCE_DIR1 = "/tmp/kvasir-dataset-v2/dyed-lifted-polyps/"
SOURCE_DIR2 = "/tmp/kvasir-dataset-v2/dyed-resection-margins/"
SOURCE_DIR3 = "/tmp/kvasir-dataset-v2/esophagitis/"
SOURCE_DIR4 = "/tmp/kvasir-dataset-v2/normal-cecum/"
SOURCE_DIR5 = "/tmp/kvasir-dataset-v2/normal-pylorus/"
SOURCE_DIR6 = "/tmp/kvasir-dataset-v2/normal-z-line/"
SOURCE_DIR7 = "/tmp/kvasir-dataset-v2/polyps/"
SOURCE_DIR8 = "/tmp/kvasir-dataset-v2/ulcerative-colitis/"

TRAINING_DIR1 = "/tmp/kvasir-dataset-v2/data/training/dyed-lifted-polyps/"
TRAINING_DIR2 = "/tmp/kvasir-dataset-v2/data/training/dyed-resection-margins/"
TRAINING_DIR3 = "/tmp/kvasir-dataset-v2/data/training/esophagitis/"
TRAINING_DIR4 = "/tmp/kvasir-dataset-v2/data/training/normal-cecum/"
TRAINING_DIR5 = "/tmp/kvasir-dataset-v2/data/training/normal-pylorus/"
TRAINING_DIR6 = "/tmp/kvasir-dataset-v2/data/training/normal-z-line/"
TRAINING_DIR7 = "/tmp/kvasir-dataset-v2/data/training/polyps/"
TRAINING_DIR8 = "/tmp/kvasir-dataset-v2/data/training/ulcerative-colitis/"

TESTING_DIR1 = "/tmp/kvasir-dataset-v2/data/testing/dyed-lifted-polyps/"
TESTING_DIR2 = "/tmp/kvasir-dataset-v2/data/testing/dyed-resection-margins/"
TESTING_DIR3 = "/tmp/kvasir-dataset-v2/data/testing/esophagitis/"
TESTING_DIR4 = "/tmp/kvasir-dataset-v2/data/testing/normal-cecum/"
TESTING_DIR5 = "/tmp/kvasir-dataset-v2/data/testing/normal-pylorus/"
TESTING_DIR6 = "/tmp/kvasir-dataset-v2/data/testing/normal-z-line/"
TESTING_DIR7 = "/tmp/kvasir-dataset-v2/data/testing/polyps/"
TESTING_DIR8 = "/tmp/kvasir-dataset-v2/data/testing/ulcerative-colitis/"

split_size = .8
split_data(SOURCE_DIR1, TRAINING_DIR1, TESTING_DIR1, split_size)
split_data(SOURCE_DIR2, TRAINING_DIR2, TESTING_DIR2, split_size)
split_data(SOURCE_DIR3, TRAINING_DIR3, TESTING_DIR3, split_size)
split_data(SOURCE_DIR4, TRAINING_DIR4, TESTING_DIR4, split_size)
split_data(SOURCE_DIR5, TRAINING_DIR5, TESTING_DIR5, split_size)
split_data(SOURCE_DIR6, TRAINING_DIR6, TESTING_DIR6, split_size)
split_data(SOURCE_DIR7, TRAINING_DIR7, TESTING_DIR7, split_size)
split_data(SOURCE_DIR8, TRAINING_DIR8, TESTING_DIR8, split_size)


In [7]:
print(len(os.listdir(TRAINING_DIR1)))
print(len(os.listdir(TRAINING_DIR2)))
print(len(os.listdir(TRAINING_DIR3)))
print(len(os.listdir(TRAINING_DIR4)))
print(len(os.listdir(TRAINING_DIR5)))
print(len(os.listdir(TRAINING_DIR6)))
print(len(os.listdir(TRAINING_DIR7)))
print(len(os.listdir(TRAINING_DIR8)))

print(len(os.listdir(TESTING_DIR1)))
print(len(os.listdir(TESTING_DIR2)))
print(len(os.listdir(TESTING_DIR3)))
print(len(os.listdir(TESTING_DIR4)))
print(len(os.listdir(TESTING_DIR5)))
print(len(os.listdir(TESTING_DIR6)))
print(len(os.listdir(TESTING_DIR7)))
print(len(os.listdir(TESTING_DIR8)))

800
800
800
800
800
800
800
800
200
200
200
200
200
200
200
200


In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

image_width, image_height = 150,150

TRAINING_DIR = "/tmp/kvasir-dataset-v2/data/training/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/tmp/kvasir-dataset-v2/data/testing/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(image_width,image_height),
	class_mode='categorical',
  batch_size=128
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(image_width,image_height),
	class_mode='categorical',
  batch_size=128
)

# model = tf.keras.models.Sequential([
#     # Note the input shape is the desired size of the image 150x150 with 3 bytes color
#     # This is the first convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     # The second convolution
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The third convolution
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # The fourth convolution
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # Flatten the results to feed into a DNN
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.5),
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(8, activation='softmax')
# ])


# model.summary()

# model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=100, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)




Found 6400 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Epoch 1/100
20/20 [==============================] - 56s 3s/step - loss: 0.3377 - accuracy: 0.8641 - val_loss: 0.3233 - val_accuracy: 0.8828
Epoch 2/100
20/20 [==============================] - 55s 3s/step - loss: 0.3557 - accuracy: 0.8566 - val_loss: 0.3293 - val_accuracy: 0.8542
Epoch 3/100
20/20 [==============================] - 55s 3s/step - loss: 0.3293 - accuracy: 0.8711 - val_loss: 0.3480 - val_accuracy: 0.8594
Epoch 4/100
20/20 [==============================] - 54s 3s/step - loss: 0.3255 - accuracy: 0.8684 - val_loss: 0.4180 - val_accuracy: 0.8411
Epoch 5/100
20/20 [==============================] - 55s 3s/step - loss: 0.3429 - accuracy: 0.8555 - val_loss: 0.3238 - val_accuracy: 0.8750
Epoch 6/100
20/20 [==============================] - 55s 3s/step - loss: 0.3469 - accuracy: 0.8668 - val_loss: 0.3668 - val_accuracy: 0.8464
Epoch 7/100
20/20 [==============================] - 54s 3s/step - los

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()